In [ ]:
import stomp
############## Setup the beamline ########################################
from dodal.beamlines.p99 import *
from bluesky.run_engine import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
import bluesky.plans as bp
from bluesky import plan_stubs as bps
from databroker import Broker
db = Broker.named('temp')
RE= RunEngine()
RE.subscribe(bec)
RE.subscribe(db.insert)
import matplotlib.pyplot as plt
import json
######## These are all the device available on p99.########################
sample_stage = sample_stage(connect_immediately = True)
andor_point = andor2_point (connect_immediately = True)
from itertools import count as cnt
scan_no = cnt(0,1)

In [ ]:
def save_nexus_data(name, doc,counter=scan_no):
    docs: dict[str, list[dict]] = {}
    docs["name"] = name
    if name == "start":
        counter = next(scan_no)
        doc.update({"instrument": "p99"})
        doc.update( {"instrument_session": "cm12345-1"})
        doc.update({"scan_id": counter})
        file_name = f"p99-{counter}"
        doc.update({"data_session": file_name})
    docs["doc"] = doc
    docs["task_id"] = doc["uid"]
    user_encode_data = json.dumps(docs).encode('utf-8')
    conn.send(body=user_encode_data, destination=DESTINATION, headers={'persistent': 'true'})

In [ ]:
HOST = HOST # Replace with your HOST
PORT = 61613
USER = USER # Replace with your username
PASSWORD = PASSWORD # Replace with your password
DESTINATION = '/queue/public.worker.event.nexus'
conn = stomp.Connection([(HOST, PORT)])

In [ ]:
RE(bp.scan([sample_stage.y],sample_stage.x, 0.1,0.3,5),(save_nexus_data))